In [ ]:
import json

In [ ]:
with open('trees.json') as fobj:
    data = json.load(fobj)

In [ ]:
data['schema']

In [ ]:
data_subset = data['data'][0:500]

In [ ]:
# data_subset

In [ ]:
import geojson

from ipyleaflet import Map, GeoJSON

from traitlets import Float

m = Map(center=[48.8566, 2.3522])

geojson_data = geojson.Feature(
    geometry=geojson.MultiPoint(
        coordinates=[
            [tree['lng'], tree['lat']]
            for tree in data_subset
        ],
    )
)

point_style = {"color": "green", "radius": 3}

geojson_layer = GeoJSON(data=geojson_data, point_style=point_style)

m.add_layer(geojson_layer)

m

In [ ]:
import pandas as pd
from py2vega.functions.color import rgb
from ipywidgets import Text, VBox
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr

df = pd.DataFrame(data_subset).drop("index", axis=1)
df = df[sorted(df.columns)]

# Get list of possible tree species and kinds for the ordinal color scales
species = list(
    set(
        [
            tree["Species"]
            for tree in data["data"]
            if tree["Species"] is not None
        ]
    )
)
kinds = list(
    set([tree["Kind"] for tree in data["data"] if tree["Kind"] is not None])
)

# Text box for highlighting tree names
highlight_box = Text(description="Highlight: ", value="Platane")
name_background_color = Expr(
    "'lightgreen' if '{}' in cell.value else default_value".format(
        highlight_box.value
    )
)


def on_textbox_changed(*args, **kwargs):
    name_background_color.value = (
        "'lightgreen' if '{}' in cell.value else default_value".format(
            highlight_box.value
        )
    )


highlight_box.observe(on_textbox_changed, "value")


def circumference_color(cell):
    color_value = 255 if cell.value > 300 else (cell.value / 300) * 255
    return rgb(0, color_value, color_value)


renderers = {
    "French_name": TextRenderer(background_color=name_background_color),
    "Height_m": BarRenderer(
        bar_value=LinearScale(min=0, max=30),
        bar_color=ColorScale(min=0, max=30, scheme="viridis"),
        horizontal_alignment="center",
    ),
    "Species": TextRenderer(
        text_color="black", background_color=OrdinalColorScale(domain=species)
    ),
    "Kind": TextRenderer(
        text_color="black", background_color=OrdinalColorScale(domain=kinds)
    ),
    "Type": TextRenderer(missing="Unspecified"),
    "Circumference_cm": BarRenderer(
        bar_value=LinearScale(min=0, max=300),
        bar_color=Expr(circumference_color),
        horizontal_alignment="center",
    ),
}

datagrid = DataGrid(
    df, base_row_size=32, base_column_size=90, renderers=renderers
)
VBox((highlight_box, datagrid))

In [ ]:
datagrid.selection_mode = 'row'

In [ ]:
from ipyleaflet import Marker

marker = Marker()

In [ ]:
def on_click(cell_info):
    selected_row = cell_info['row']

    selected_tree = data_subset[selected_row]

    marker.location = (selected_tree['lat'], selected_tree['lng'])

In [ ]:
datagrid.on_cell_click(on_click)

In [ ]:
m.add_layer(marker)

In [ ]:
# data_subset